In [ ]:
!pip install accelerate -U
!pip install transformers[torch]


In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# 加载数据
data_path = '/content/drive/MyDrive/data/'
train_df = pd.read_csv(data_path + 'train.csv')
test_df = pd.read_csv(data_path + 'test.csv')

# 只保留 'reviewbody' 和 'star' 列，并重命名 'reviewbody' 列为 'review'
train_df = train_df[['review', 'star']]
train_df = train_df.head(12000)
test_df = test_df[['review', 'star']]
test_df = test_df.head(1000)

# 数据预处理函数
def preprocess_data(df):
    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

    def tokenize_function(examples):
        return tokenizer(examples['review'], truncation=True, padding=True)

    dataset = Dataset.from_pandas(df)
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    return tokenized_dataset

# 预处理训练集和测试集
train_dataset = preprocess_data(train_df)
test_dataset = preprocess_data(test_df)

# 添加标签
def add_labels(example):
    example['labels'] = example['star']
    return example

train_dataset = train_dataset.map(add_labels)
test_dataset = test_dataset.map(add_labels)

# 加载预训练的BERT模型
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=1)  # num_labels=1 表示回归任务

# 定义训练参数
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',  # 用于存储日志
    logging_steps=10,
)

# 创建Trainer实例
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# 开始训练
trainer.train()

# 评估模型
results = trainer.evaluate()
print(results)


In [ ]:
!pip install tensorflow tensorboard


In [ ]:
import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import matplotlib.pyplot as plt

# 定义函数来加载和提取损失数据
def load_and_extract_losses(log_file):
    event_acc = EventAccumulator(log_file)
    event_acc.Reload()

    tags = event_acc.Tags()['scalars']
    print(f"All tags for {log_file}: {tags}")

    train_losses = []
    eval_losses = []

    if 'train/loss' in tags:
        train_events = event_acc.Scalars('train/loss')
        train_losses = [event.value for event in train_events]
    else:
        print(f"No 'train/loss' tag found in {log_file}.")

    if 'eval/loss' in tags:
        eval_events = event_acc.Scalars('eval/loss')
        eval_losses = [event.value for event in eval_events]
    else:
        print(f"No 'eval/loss' tag found in {log_file}.")

    return train_losses, eval_losses

# 加载并提取第一个事件文件的损失数据
log_file1 = './logs/events.out.tfevents.1719327193.e90606a6042e.9513.0'
train_losses1, eval_losses1 = load_and_extract_losses(log_file1)

# 加载并提取第二个事件文件的损失数据
log_file2 = './logs/events.out.tfevents.1719329449.e90606a6042e.9513.1'
train_losses2, eval_losses2 = load_and_extract_losses(log_file2)

# 打印一部分损失值，检查数据是否正确提取
print(f"Train losses (File 1): {train_losses1[:10]}")
print(f"Eval losses (File 1): {eval_losses1[:10]}")
print(f"Train losses (File 2): {train_losses2[:10]}")
print(f"Eval losses (File 2): {eval_losses2[:10]}")

# 绘制损失曲线
plt.figure(figsize=(16, 8))

# 绘制第一个文件的损失曲线
plt.subplot(1, 2, 1)
if train_losses1:
    plt.plot(train_losses1, label='Training Loss')
plt.plot(eval_losses1, label='Validation Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training and Validation Loss (File 1)')
plt.legend()
plt.grid(True)

# 绘制第二个文件的损失曲线
plt.subplot(1, 2, 2)
if train_losses2:
    plt.plot(train_losses2, label='Training Loss')
plt.plot(eval_losses2, label='Validation Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training and Validation Loss (File 2)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()
